In [4]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [5]:
this_month='1711'
data_path='../data/'+this_month

In [6]:
df=pd.read_csv(data_path+'/cleaned.csv')
df.shape

(12145854, 15)

In [7]:
df=df[~((df.year==2014) & (df.mm<int(this_month[2:4])))]
df.shape

(9650796, 15)

In [8]:
df=df[(df['year']<=2017) & (df['year']>=2014)]
df.shape

(9650796, 15)

In [9]:
unique_subjects=pd.read_csv('../store/unique_subjects.csv',header=None)
unique_subjects=unique_subjects[0].values

In [10]:
with open('../store/issn_subject.json') as data_file:    
    issn_subject = json.load(data_file)

In [11]:
df2=df[['doi','main_issn','citation']]

In [12]:
join_subjects=df2['main_issn'].apply(lambda x: ';'.join(issn_subject[x]) if x in issn_subject else '')

In [13]:
df2['subjects']=join_subjects.values

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
df2['main_issn'].apply(lambda x: ';'.join(issn_subject[x]) if x in issn_subject else '').shape

(9650796L,)

In [15]:
unique_subjects=[]
all_subjects=df2['subjects'].values
for journal_subjects in all_subjects:
    subjects=journal_subjects.split(';')
    for subject in subjects:
        if subject not in unique_subjects: unique_subjects.append(subject)

In [16]:
len(unique_subjects)

327

In [21]:
top_articles=[]
counter=0
for subject in unique_subjects:
    if subject=='':
        df_subject=df2.sort_values('citation', ascending= False)[:20]
    else:
        df_subject=df2[df2.subjects.apply(lambda x: subject in x.split(';'))].sort_values('citation', ascending= False)[:20]
    top_dois=df_subject[['doi','main_issn','citation']].to_dict('list')
    zips=zip(top_dois['doi'],top_dois['main_issn'],top_dois['citation'])
    items=[];
    for doi,issn,citation in zips:
        item={}; item_cleaned={}
        url='http://api.crossref.org/works/'+doi
        while True:
            try:
                response = urllib.urlopen(url).read()    
            except Exception,e: 
                print str(e)
                time.sleep(1)
                continue
            break
        json_result=json.loads(response)
        item = json_result['message']
        year_print=[9999,99,99];year_online=[9999,99,99];
        if 'title' in item:
            title=item['title'][0]
        if 'published-print' in item:
            year_print=item['published-print']['date-parts'][0]                
        if 'published-online' in item:
            year_online=item['published-online']['date-parts'][0]
        year=min(year_print,year_online)
        if 'DOI' in item: doi=item['DOI']
        if 'container-title' in item: journal=','.join(item['container-title'])
        if 'publisher' in item: pub=item['publisher']; pub=''if pub==None else pub
        if 'references-count' in item: ref=str(item['references-count'])
        authors=[]
        if 'author' in item: 
            for author in item['author']:
                if 'given' in author:
                    author_name=author['given']
                if 'family' in author:
                    author_name+=' '+author['family']
                authors.append(author_name)
        if len(authors)>5: authors=authors[:5]
        if len(year)>1 :
            month=str(year[1]) 
        else: month=''
        item_cleaned={'@id':doi,
                      'e:title':title,
                      'e:year':str(year[0])+'-'+month,
                      'e:cite':np.asscalar(citation),
                      'e:issn':issn, 
                      'e:journal':journal,
                      'e:pub':pub,
                      'e:authors':authors,
                      'e:subject':subject,
                      'e:month':this_month} 
        if item_cleaned not in top_articles:
            top_articles.append(item_cleaned)
    print counter,
    counter+=1            

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [23]:
with open('..\web\cats.json') as data_file:    
    cat_subject=json.load(data_file)

In [24]:
sub_cat={}
for cat in cat_subject.keys():
    for sub in cat_subject[cat].split(';'):
        sub_cat[sub]=cat

In [35]:
top_articles_new=[]
for a in top_articles:
    a['e:subject']=a['e:subject'].strip()
    a['e:category']=''
    subject=a['e:subject']
    if subject!='':
        cat=sub_cat[subject]
        a['e:category']=sub_cat[subject]
    top_articles_new.append(a)

In [37]:
with open('../jsonld/'+this_month+'/top_articles.jsonld', 'w') as fp:
    json.dump(top_articles_new, fp)    

for subject in unique_subjects:
    df_lic=df2[df2.subjects.apply(lambda x: subject in x.split(';'))]
    top_lics=df_lic[['lic','count']].groupby('lic').count().sort_values('count',ascending=False).head().reset_index().to_dict('list')
    zips=zip(top_lics['lic'],top_lics['count'])
    total_articles=str(df_lic.shape[0])
    items=[{'count':total_articles,'lic':'all'}];
    for lic,count in zips:
        item={}; 
        item={'lic':lic,'count':str(count)}
        items.append(item)  
    with open('../jsonld/lic_'+subject+'.json', 'w') as fp:
        json.dump(items, fp)    